In [4]:
import os

positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('../opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
read_lexicon()
op_set = positive_lexicon + negative_lexicon

In [7]:
import pandas as pd

sf = pd.DataFrame(columns=['reviewID', 'sentenceID', 'review', 'target', 'category', 'polarity'])
df = pd.read_csv('../dataset/res16_nan_all.csv')
categories = {}
count = 0
f_count, a_count, s_count, p_count = 0, 0, 0, 0
for index in range(0, len(df)):
    category = df['category'][index].split(',')
    ncat = []
    sent_temp = []
    target = df['target'][index]
    
    is_food, is_service, is_ambience, is_price = False, False, False, False
    for c in category:
        if c == "FOOD":
            is_food = True
        elif c == "PRICES":
            is_price = True
        elif c == "AMBIENCE":
            is_ambience = True
        elif c == "SERVICE":
            is_service = True
        if c not in sent_temp:
            try:
                categories[c] += 1
            except:
                categories[c] = 1
        sent_temp.append(c)
        
    if is_price:
        sf = sf.append({'reviewID': df['reviewID'][index],
                            'sentenceID': df['sentenceID'][index],
                            'review': df['review'][index],
                            'target': df['target'][index],
                            'category': df['category'][index],
                            'polarity': df['polarity'][index]}, ignore_index=True)

        if is_food:
            f_count += 1
        if is_ambience:
            a_count += 1
        if is_service:
            s_count += 1
        if is_price:
            p_count += 1
            
for index in range(0, len(df)):
    category = df['category'][index].split(',')
    ncat = []
    sent_temp = []
    target = df['target'][index]
    sentence =  df['review'][index].lower().replace('  ', ' ').strip()
    tokenized = sentence.split(' ')
    
    is_food, is_service, is_ambience, is_price = False, False, False, False
    for c in category:
        if c == "FOOD":
            is_food = True
        elif c == "PRICES":
            is_price = True
        elif c == "AMBIENCE":
            is_ambience = True
        elif c == "SERVICE":
            is_service = True
        if c not in sent_temp:
            try:
                categories[c] += 1
            except:
                categories[c] = 1
        sent_temp.append(c)
    
#     is_dp = False
#     print(sentence)
#     for token in tokenized:
#         if token in op_set:
#             is_dp = rule(sentence, token)
#             break
            
    if not is_price: #and is_dp:
        if f_count > 129 and is_food:
            continue
        if a_count > 129 and is_ambience:
            continue
        if s_count > 129 and is_service:
            continue

        sf = sf.append({'reviewID': df['reviewID'][index],
                            'sentenceID': df['sentenceID'][index],
                            'review': df['review'][index],
                            'target': df['target'][index],
                            'category': df['category'][index],
                            'polarity': df['polarity'][index]}, ignore_index=True)

        if (is_food and is_service) and is_ambience:
            f_count += 1
            s_count += 1
            a_count += 1
        elif (is_food and is_service) and not is_ambience:
            f_count += 1
            s_count += 1
        elif (is_food and is_ambience) and not is_service:
            f_count += 1
            a_count += 1
        elif (is_service and is_ambience) and not is_food:
            s_count += 1
            a_count += 1
        elif (is_food) and not (is_price):
            f_count += 1
        elif (is_ambience) and not (is_price):
            a_count += 1
        elif (is_service) and not (is_price):
            s_count += 1            

sf.to_csv("dataset/res16_baru.csv")
sf.to_excel("dataset/res16_baru.xlsx")

In [8]:
import pandas as pd

df = pd.read_csv('dataset/res16_baru.csv')
categories = {}
count = 0
for index, cat in enumerate(df['category']):
    count+=1
    category = cat.split(',')
    ncat = []
    sent_temp = []
    target = df['target'][index]
    for c in category:
        x = c#.split('#')[0]
        if x not in sent_temp:
            try:
                categories[x] += 1
            except:
                categories[x] = 1
        sent_temp.append(x)
print(count)

358


In [9]:
categories

{'AMBIENCE': 130, 'FOOD': 130, 'PRICES': 130, 'SERVICE': 130}

In [10]:
categories

{'AMBIENCE': 130, 'FOOD': 130, 'PRICES': 130, 'SERVICE': 130}

In [36]:
import requests
url = "http://localhost:9000/semgrex"

conj_DP = ['conj']
dep_DP = ['amod', 'prep', 'nsubj', 'csubj', 'xsubj', 'dobj', 'iobj']

def req_semgrex(params, text):
    r = requests.post(url, data = text, params=params)
    return r.json()

def rule(text, key):
    for dep in dep_DP:
        r = req_semgrex({"pattern": "{word:"+ key +"} <"+ dep +" {}"}, text)
        print('-')
        length = r['sentences'][0]['length']
        if length:
            return True
    return False

In [51]:
rule('good food', 'food')

True

In [39]:
from nltk.parse.corenlp import CoreNLPDependencyParser

parser = CoreNLPDependencyParser()


In [50]:
def rule(sentence, key):
    parse = next(parser.raw_parse(sentence))
    for (w1, dep, w2) in list(parse.triples()):
        word1 = w1[0]
        word2 = w2[0]
        if word1 == key or word2 == key:
            if dep in dep_DP:
                return True
    return False